In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Загрузка стоп-слов и стеммера
# nltk.download('stopwords')
# nltk.download('punkt')


# Получение текстовых данных из определенной колонки DataFrame
documents = (df['analysis_name'] + " " + df["best_match_synonym"]).tolist()
# documents = f"{df['analysis_name']} {df["best_match_synonym"]}"

# Загрузка стоп-слов и стеммера
stop_words = set(stopwords.words('russian'))
stemmer = SnowballStemmer('russian')

# Функция для предобработки текста
def preprocess_text(text):
    tokenizer = nltk.RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(text.lower())
    tokens = [token for token in tokens if token not in stop_words]
    stemmed_tokens = [stemmer.stem(token) for token in tokens]
    return stemmed_tokens

# Предобработка текстовых данных
processed_documents = [' '.join(preprocess_text(doc)) for doc in documents]

# Создание экземпляра TfidfVectorizer
vectorizer = TfidfVectorizer()

# Вычисление весов TF-IDF для текстовых данных 
tfidf_matrix = vectorizer.fit_transform(processed_documents)

# Ввод запроса от пользователя 
query = input("Введите запрос: ")

# Предобработка запроса 
processed_query = ' '.join(preprocess_text(query))

# Вычисление весов TF-IDF для запроса 
query_tfidf = vectorizer.transform([processed_query])

# Вычисление сходства между запросом и текстовыми данными
similarities = cosine_similarity(query_tfidf, tfidf_matrix).flatten()

# Сортировка документов по убыванию значений сходства
sorted_indexes = similarities.argsort()[::-1]

# Вывод результатов
print("Результаты поиска:")
for index in sorted_indexes:
    print(f"Документ: {documents[index]}, Сходство: {similarities[index]}")

In [10]:
df_synonyms = pd.read_csv("../data/preprocessed_data/df_reg_sys.csv")
df_analysis  = pd.read_csv("../data/preprocessed_data/df_analysis.csv")
df_synonyms

,Unnamed: 0,synonyms,analysis
0,0,анализ крови на глюкозу глюкоза в плазме или с...,глюкоза в крови glucose
1,1,анализ крови на гликированный гемоглобин glyco...,"гликированный гемоглобин hba1с, glycated hemog..."
2,2,анализ крови на фруктозамин гликозилированный ...,фруктозамин fructosamine
3,3,анализ крови на гтт пероральный глюкозотолеран...,глюкозотолерантный тест с определением глюкозы...
4,4,анализ крови на толерантность к глюкозе с с-пе...,глюкозотолерантный тест с определением глюкозы...
...,...,...,...
797,797,урогенитальный соскоб на лактобактерии,"лактобактерии, определение днк lactobаcillus s..."
798,798,рентген рентгенография,рентгенография
799,799,экг электрокардиограмма,электрокардиограмма
800,800,мрт,магнитно-резонансная томография


In [8]:
from re import A
import pandas as pd
import numpy as np
import nltk
from nltk import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import sys


In [63]:
class Model_Tfidf():


    def __init__(self, 
                stemmer: SnowballStemmer = SnowballStemmer(language="russian"), 
                tokenizer: RegexpTokenizer = RegexpTokenizer(r"\w+"),
                vectorizer: TfidfVectorizer = TfidfVectorizer()
                ):
        self.stemmer = stemmer
        self.tokenizer = tokenizer
        self.vectorizer = vectorizer
        self.tfidf_matrix = np.ndarray([])
    
    @classmethod
    def stop_words(cls) -> set:
        stop_words = set(stopwords.words('russian'))
        custom_stop_words = ("анализ", "уровень", "определение", "анализы", "консультация", "выявление")
        return set(stop_words.union(custom_stop_words))
    
    def concatenate_values(self, row):
        return ' '.join(row)

    def get_documents(self, df: pd.DataFrame) -> list:
        # Получение текстовых данных из определенных колонок DataFrame
        df['concatenated'] = df.apply(self.concatenate_values, axis=1)
        return df["concatenated"].tolist()

        # Функция для предобработки текста
    def preprocess_text(self, text):
        tokens = self.tokenizer.tokenize(text.lower())
        tokens = [token for token in tokens if token not in Model_Tfidf.stop_words()]
        stemmed_tokens = [self.stemmer.stem(token) for token in tokens]
        return stemmed_tokens

    # Предобработка текстовых данных
    def preprocessed_documents(self, df: pd.DataFrame):
        documents = self.get_documents(df)
        return [' '.join(self.preprocess_text(doc)) for doc in documents]

    # Предобработка запроса 
    def preprocessed_query(self, query:str):
        return ' '.join(self.preprocess_text(query))

    def get_similar_synonyms(self, df: pd.DataFrame, df_synonyms: pd.DataFrame):
        tfidf_synonyms = self.vectorizer.fit_transform(self.preprocessed_documents(df_synonyms[["analysis"]])) 
        tfidf_df = self.vectorizer.transform(self.preprocessed_documents(df[["analysis_name"]]))
        syn_indexes = []
        syn_similarity = []
        # Вычисление сходства между запросом и текстовыми данными
        for vec in tfidf_df:
            similarities = cosine_similarity(vec, tfidf_synonyms).flatten()
            # Сортировка документов по убыванию значений сходства
            sorted_indexes = similarities.argsort()[::-1]
            syn_indexes.append(sorted_indexes[0])
            syn_similarity.append(similarities[sorted_indexes[0]])

        df_similarity = pd.DataFrame({
                                        "best_match_synonym": df_synonyms["synonyms"].iloc[syn_indexes].values,
                                        "best_match_analysis": df_synonyms["analysis"].iloc[syn_indexes].values,
                                        "similarity": syn_similarity
                                    }, index=df.index).sort_index(ascending=False)
        
        df_result = pd.concat((df, df_similarity), axis=1)
        df_result.loc[(df_result["similarity"] < 0.64), ("best_match_synonym", "best_match_analysis")] = ""
        return df_result
    

    def get_similar_analysis(self, df: pd.DataFrame, query: str ):
        
        df.loc[(df["best_match_synonym"].isnull()), ("best_match_synonym", "best_match_analysis")] = ""
        self.tfidf_matrix = self.vectorizer.fit_transform(self.preprocessed_documents(df[["analysis_name", "best_match_synonym"]]))
        tfidf_query = self.vectorizer.transform([self.preprocessed_query(query)])

        similarities = cosine_similarity(tfidf_query, self.tfidf_matrix).flatten()
        # Сортировка документов по убыванию значений сходства
        sorted_indexes = similarities.argsort()[::-1]
        
        df["as"] = similarities            
                
        return df[["source", "analysis_name", "analysis_cost"]].iloc[sorted_indexes].loc[df["as"] >= 0.15]

In [64]:
m = Model_Tfidf()

In [44]:
df = m.get_similar_synonyms(df=df_analysis, df_synonyms=df_synonyms)

/tmp/ipykernel_4325/1005747622.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['concatenated'] = df.apply(self.concatenate_values, axis=1)
/tmp/ipykernel_4325/1005747622.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['concatenated'] = df.apply(self.concatenate_values, axis=1)


In [46]:
df.loc[df["similarity"] > 0.7]

,Unnamed: 0,index,source,chapter,analysis_name,analysis_cost,analysis_comment,best_match_synonym,best_match_analysis,similarity
5,5,5,6gkb.by,Лабораторная диагностика,общий анализ мочи пункты 1 2 + 2 1 9 + 2 1 4 1...,5.51,NaN,клинический анализ мочи оам complete urinalysi...,анализ мочи общий анализ мочи с микроскопией о...,0.702894
63,63,63,6gkb.by,Лабораторная диагностика,подсчет тромбоцитов крови в окрашеных мазках п...,3.34,NaN,"анализ крови на тромбоциты тромбоциты, микроск...","тромбоциты, микроскопия подсчет в окрашенном м...",0.862644
84,84,84,6gkb.by,Лабораторная диагностика,са 242*,16.99,NaN,анализ крови на са-242 углеводный антиген 242 ...,"ca-242 углеводный антиген са-242, опухолевый м...",0.705643
87,87,87,6gkb.by,Лабораторная диагностика,определение кальцитонина,18.09,NaN,анализ крови на кальцитонин тиреокальцитонин т...,кальцитонин calcitonin,0.707107
95,95,95,6gkb.by,Лабораторная диагностика,холестерин высокой плотности лпвп hdl биохимич...,1.65,NaN,липопротеиды высокой плотности лпвп лвп хс лпв...,холестерин-лпвп холестерин липопротеинов высок...,0.757359
...,...,...,...,...,...,...,...,...,...,...
4893,4893,4977,Lode.by,NaN,фруктозамин fructosamine,3.99,Фруктозамин - комплекс глюкозы с белками крови...,анализ крови на фруктозамин гликозилированный ...,фруктозамин fructosamine,1.000000
4909,4909,4993,Lode.by,NaN,выявление антител класса igg к yersinia entero...,10.20,Выявление антител класса IgG к Yersinia Entero...,специфические иммуноглобулины класса igg к бак...,антитела класса igg к антигенам yersinia ente...,0.806810
4916,4916,5000,Lode.by,NaN,"гонококк , определение днк neisseria gonorrhoe...",11.26,Тест для определения генетического материала (...,"гонококк, определение днк в моче возбудитель г...","гонококк, определение днк neisseria gonorrhoea...",0.794828
4918,4918,5002,Lode.by,NaN,антитела igm к бета2 - гликопротеину i,13.23,Важный показатель в диагностике антифосфолипид...,анализ крови на igm антитела к бета-2-гликопро...,"антитела к бета-2-гликопротеину, igm anti-beta...",0.708758


In [25]:
a = m.get_documents(df_analysis[["analysis_name"]])
s = m.get_documents(df_synonyms[["analysis"]])

/tmp/ipykernel_4325/1129943349.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['concatenated'] = df.apply(self.concatenate_values, axis=1)
/tmp/ipykernel_4325/1129943349.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['concatenated'] = df.apply(self.concatenate_values, axis=1)


In [62]:
m.preprocessed_documents(df[["analysis_name", "best_match_synonym"]])

TypeError: sequence item 1: expected str instance, float found

In [65]:
m.get_similar_analysis(df, "экб")

/tmp/ipykernel_4325/185580246.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['concatenated'] = df.apply(self.concatenate_values, axis=1)


,source,analysis_name,analysis_cost
4343,Lode.by,"эозинофильный катионный белок экб, еср",56.72
2124,Synevo.by,эозинофильный катионный белок ecp 08-094,56.72


In [61]:
df.loc[df["best_match_synonym"].isna()]

,Unnamed: 0,index,source,chapter,analysis_name,analysis_cost,analysis_comment,best_match_synonym,best_match_analysis,similarity
4818,4818,4902,Lode.by,NaN,посев раневого отделяемого и тканей на микрофл...,29.6,Микробиологическое иследование на определение ...,NaN,посев раневого отделяемого и тканей на микрофл...,0.799534
